In [ ]:
# Model with two input arrays
# didn't work for some reason
import keras
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback


# REMOVE ARRAY_1
input1 = Input(shape=(29,), name='material_type')
input2 = Input(shape=(29,), name='material_weight')
merged = Concatenate(axis=1)([input1, input2])
dense1 = Dense(300,input_dim=2, activation="relu", use_bias=True)(merged)
output = Dense(123, activation="sigmoid")(dense1)
    
model =  Model(inputs=[input1, input2], outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

#tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
train_array = [X1_TRAIN[0:10000], X2_TRAIN[0:10000]]
test_array = [X1_TRAIN[10000:12745], X2_TRAIN[10000:12745]]
train_Y = LABELS[0:10000]
test_Y = LABELS[10000:12745]

model.fit([X1_TRAIN,X2_TRAIN], LABELS, batch_size=64, epochs=5, validation_split=0, verbose=2) 

y_pred1 = model.predict([X1_TEST,X2_TEST], verbose=0)
y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.3] = 1 
 
#from sklearn.metrics import f1_score
#scores = f1_score(test_Y, y_pred1 , average="micro")
#scores

c = 0
for cd in LABELS:
     if cd[0] == 1:
            c += 1

print(len(LABELS))
print(c)

c = 0
for cd in y_pred:
     if cd[0] == 1:
            c += 1
print(len(y_pred))
print(c)

#12746
#4901
#8397
#5002

In [ ]:
3 * 400

    input1 = Input(shape=(29,) , name='material_weight')
    dense1 = Dense(400, activation="relu")(input1)
    bn1 = BatchNormalization()(dense1)
    
    dense2 = Dense(400, activation="relu")(bn1)##########
    bn2 = BatchNormalization()(dense2)#
    dense3 = Dense(400, activation="relu")(bn2)#
    bn3 = BatchNormalization()(dense3)##################
    
    output = Dense(123, activation="sigmoid")(bn3)
    model_CV =  Model(inputs=input1, outputs=output)

    model_CV.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['categorical_accuracy'])

    model_CV.fit(X[train], Y[train], batch_size=64, epochs=35,verbose=0 )
    
#   0.360 (+/- 0.008)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import f1_score

classif = OneVsRestClassifier(model_test)
classif.fit(X, Y)
predicted_values = classif.predict(X_val) 
print(f1_score(Y_val, predicted_values , average="micro"),'OneVsRestClassifier')  # LABELS

classifierBinaryRelevance = BinaryRelevance(model_test)
classifierBinaryRelevance.fit(X, Y)
predicted_values = classifierBinaryRelevance.predict(X_val) 
print(f1_score(Y_val, predicted_values , average="micro"),'BinaryRelevance')  # LABELS

classifierChain = ClassifierChain(model_test)
classifierChain.fit(X, Y)
predicted_values = classifierChain.predict(X_val) 
print(f1_score(Y_val, predicted_values , average="micro"),'ClassifierChain')  # LABELS

classifierLabelPowerset = LabelPowerset( model_test )
classifierLabelPowerset.fit(X, Y)
predicted_values = classifierLabelPowerset.predict(X_val) 
print(f1_score(Y_val, predicted_values , average="micro"),'LabelPowerset')  # LABELS

In [ ]:
model_3 = LabelPowerset(classifier=model_test , require_dense=[True,False])
model_3.fit(X2_TRAIN, LABELS)
y_pred2 = model_3.predict(X2_TEST)
y_pred2[0]

In [ ]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=3)

#y_train = lil_matrix(y_train).toarray()
#x_test = lil_matrix(x_test).toarray()# train
classifier_new.fit(X, Y)# predict

predicted_values = classifier_new.predict(X_val) 
print(f1_score(Y_val, predicted_values , average="micro"))  # LABELS
predicted_values[0]

In [ ]:
#model_2 = ClassifierChain( model_test, require_dense = [False, True])
#model_2.fit( X2_TRAIN, LABELS)

In [ ]:
# 10-fold cross validation
from sklearn.model_selection import KFold
import keras
from keras.models import Model
from keras.layers import Input, Dense, SpatialDropout1D, Concatenate,BatchNormalization
from keras.callbacks import Callback
import numpy
#early_stop = keras.callbacks.EarlyStopping(monitor='acc', min_delta=0, patience=10, verbose=0, mode='auto', baseline=None)
#reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='acc', factor=0.2, patience=4, min_lr=0.001)
from sklearn.metrics import f1_score

seed = 7
numpy.random.seed(seed)

X = X2_TRAIN
Y = LABELS

kfold = KFold(n_splits=10,shuffle=True,random_state=seed)

cvscores = []
for train, test in kfold.split(X, Y):
    input1 = Input(shape=(29,) , name='material_weight')
    dense1 = Dense(400, activation="relu")(input1)
    bn1 = BatchNormalization()(dense1)
    dense2 = Dense(400, activation="relu")(bn1)
    bn2 = BatchNormalization()(dense2)
    dense3 = Dense(400, activation="relu")(bn2)
    bn3 = BatchNormalization()(dense3)
    dense4 = Dense(400, activation="relu")(bn3)
    bn4 = BatchNormalization()(dense4)
    output = Dense(123, activation="sigmoid")(bn4)

    model_CV = Model(inputs=input1, outputs=output)

    model_CV.compile(loss='binary_crossentropy', 
                     optimizer='Adam',
                     metrics=['accuracy'])

    model_CV.fit(X[train], Y[train], batch_size=64, epochs=35,verbose=0 ) #, callbacks=[early_stop,reduce_lr]
    # 38 60
    y_pred1 = model_CV.predict(X[test], verbose=0)
    y_pred = np.zeros(y_pred1.shape)
    y_pred[y_pred1>0.3] = 1 
    
    scores = f1_score(Y[test], y_pred , average="micro")
    print("%s: %.3f%%" % ('f1 micro-average: ', scores))
    cvscores.append(scores)

print("%.3f%% (+/- %.3f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
y_pred1 = model_2.predict(X2_TRAIN, verbose=0)

y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.3] = 1 

print(f1_score(LABELS, y_pred, average="micro"))
#0.284 - 0.3

s = y_pred1
labels = np.zeros(s.shape)
labels[s>0.3] = 1 
good = 0
bad = 0
for d in range(len(LABELS)):
    aaa =(labels[d] == LABELS[d])
    if aaa.all() is True:
        good += 1
    else:
        bad += 1
        
print(good, bad)
#3184 9562  dense 123

In [ ]:
import talos
import keras
from keras.layers import Input, Dense, Dropout,BatchNormalization
from keras.models import Model
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X2_TRAIN, LABELS, test_size=0.2, random_state=52, shuffle = True)

from keras.callbacks import EarlyStopping
early = EarlyStopping(monitor='val_f1score', min_delta=0.0001, patience=8, mode='max')

def telco_churn(x_train, y_train, x_val, y_val, params):
    input_layer = Input(shape=(29,))
    hidden_layer1 = Dense(400, activation='relu')(input_layer)
    bn1 = BatchNormalization()(hidden_layer1)
    hidden_layer2 = Dense(400, activation='relu')(bn1)
    bn2 = BatchNormalization()(hidden_layer2)
    hidden_layer3 = Dense(400, activation='relu')(bn2)
    bn3 = BatchNormalization()(hidden_layer3)
    hidden_layer4 = Dense(400, activation='relu')(bn3)
    bn4 = BatchNormalization()(hidden_layer4)
    output = Dense(123, activation="sigmoid")(bn4)

    model = Model(inputs=input_layer, outputs=output)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[talos.utils.metrics.f1score])

    out = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val], epochs=params['epochs'],
                    batch_size=64,verbose=0,callbacks=[early])
   
    return out, model

#keras.layers.LeakyReLU(alpha=0.3)
p = { 'epochs': [1,2,3,4,5,6,7,8,10,15,20,30,40,50,60], }
#[logcosh, binary_crossentropy],

scan_object = talos.Scan(x=x_train, y=y_train, x_val=x_val, y_val=y_val, params=p, 
                         model=telco_churn,experiment_name='123')

d = scan_object.best_model(metric='f1score', asc=False)
scan_object.data.sort_values(by='val_f1score', ascending=False).to_csv('HyperOpt42.csv')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier# Using pipeline for applying logistic regression and one vs rest classifier

LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X, y)

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
y_pred1 = model_2.predict(X2_TEST, verbose=0)

y_pred = np.zeros(y_pred1.shape)
y_pred[y_pred1>0.3] = 1 

c = 0
for cd in LABELS:
     if cd[0] == 1:
            c += 1
print(len(LABELS))
print(c)

c=0
for cd in y_pred:
     if cd[0] == 1:
            c += 1
print(len(y_pred))
print(c)

#12746
#4901
#8397
#5002